## Final processing before modeling

#### Inputs:
- ts_data_pp01_chunked_moist_weather.pkl
- chunk_data_pp01.pkl

#### Outputs:
- df_train01.csv
- df_test01.csv
- df_chunk_train01.csv
- df_chunk_test01.csv
- df_all01.csv
- df_chunk01.csv

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import pickle
import sys
from importlib import reload
import logging

%matplotlib inline

In [2]:
sys.path.append('..')

In [3]:
df_all = pd.read_pickle('../data/ts_data_pp01_chunked_moist_weather.pkl')
df_chunk = pd.read_pickle('../data/chunk_data_pp01.pkl')

#### Keep only the chunks that have more than 100 hours of data

In [4]:
ds_chuncounts = df_all.groupby('ChunkId').ChunkId.count()

In [5]:
df_all = df_all[df_all.ChunkId.isin(list(ds_chuncounts[ds_chuncounts > 100].index))]

#### Remove multiply defined chunks

In [6]:
df_chunk = df_all.groupby('ChunkId').agg({'Latitude': 'first', 'Longitude': 'first', 'LocId': 'first'}).reset_index()

In [7]:
df_all.shape, df_chunk.shape

((2332391, 18), (4912, 4))

In [8]:
df_all.LocId.nunique(), df_all.ChunkId.nunique()

(1363, 4912)

#### Remove chunks that have missing temperatures

In [9]:
df_all['IsTmpNull'] = df_all.temperature.isnull()

In [10]:
ds_chunks_full = df_all.groupby('ChunkId').IsTmpNull.sum() == 0
chunks_full = list(ds_chunks_full[ds_chunks_full == True].index)
df_all = df_all[df_all.ChunkId.isin(chunks_full)]

In [11]:
df_chunk = df_all.groupby('ChunkId').agg({'Latitude': 'first', 'Longitude': 'first', 'LocId': 'first'}).reset_index()

In [12]:
df_all.shape, df_chunk.shape

((1781155, 19), (3967, 4))

In [13]:
df_all.LocId.nunique(), df_all.ChunkId.nunique()

(1274, 3967)

In [14]:
df_chunk_sub = df_chunk.sample(50).copy()
df_all_sub = df_all[df_all.ChunkId.isin(df_chunk_sub.ChunkId.values)].copy()

#### Fill null data with means (first try chunk-mean, if all null, then try location-mean, if still null, use all data)

In [16]:
df_all.isnull().sum()

LocId                         0
Datetime                      0
Moisture                      0
Latitude                      0
Longitude                     0
ChunkBegin                    0
ChunkId                       0
precip_intensity          19433
precip_probability        19433
precip_type             1060915
temperature                   0
apparent_temperature          0
dew_point                     0
humidity                      0
wind_speed                  117
wind_gust                 27931
cloud_cover               20424
visibility                39250
IsTmpNull                     0
dtype: int64

In [17]:
df_all.precip_type.value_counts()

rain    691879
snow     28361
Name: precip_type, dtype: int64

In [18]:
df_all['Day'] = df_all.Datetime.apply(lambda x: x.day)
df_all['Hour'] = df_all.Datetime.apply(lambda x: x.hour)
df_all['Month'] = df_all.Datetime.apply(lambda x: x.month)
df_all['Weekday'] = df_all.Datetime.apply(lambda x: x.weekday())

In [19]:
def fill_nulls(colnow):
    
    print(colnow)
    print(df_all[colnow].isnull().sum())
    df_all[colnow] = df_all.groupby('ChunkId')[colnow].transform(lambda x: x.fillna(x.mean()))
    print(df_all[colnow].isnull().sum())
    df_all[colnow] = df_all.groupby('LocId')[colnow].transform(lambda x: x.fillna(x.mean()))
    print(df_all[colnow].isnull().sum())
    df_all[colnow] = df_all[colnow].fillna(df_all[colnow].mean())
    print(df_all[colnow].isnull().sum())

In [21]:
fill_nulls('precip_intensity')

precip_intensity
19433
17525
2370
0


In [22]:
fill_nulls('precip_probability')

precip_probability
19433
17525
2370
0


In [23]:
fill_nulls('wind_speed')

wind_speed
117
0
0
0


In [24]:
fill_nulls('wind_gust')

wind_gust
27931
12568
1297
0


In [25]:
fill_nulls('cloud_cover')

cloud_cover
20424
0
0
0


In [26]:
fill_nulls('visibility')

visibility
39250
39169
7569
0


In [28]:
df_chunk.shape

(3967, 4)

#### Split data as train and test

In [29]:
df_chunk.LocId.nunique()

1274

In [30]:
n_locs_train = 1000
loc_ids = df_chunk.LocId.unique()

inds_rand = np.random.permutation(np.arange(len(loc_ids)))
loc_ids_train = loc_ids[inds_rand[:n_locs_train]]
loc_ids_test = loc_ids[inds_rand[n_locs_train:]]

df_chunk_train = df_chunk[df_chunk.LocId.isin(loc_ids_train)]
df_chunk_test = df_chunk[df_chunk.LocId.isin(loc_ids_test)]

df_train = df_all[df_all.ChunkId.isin(df_chunk_train.ChunkId.values)]
df_test = df_all[df_all.ChunkId.isin(df_chunk_test.ChunkId.values)]

In [33]:
df_chunk_train.LocId.nunique(), df_chunk_test.LocId.nunique()

(1000, 274)

In [32]:
df_chunk_train.shape, df_chunk_test.shape

((3086, 4), (881, 4))

In [31]:
df_train.shape, df_test.shape

((1392707, 23), (388448, 23))

In [34]:
df_train.to_csv('../data/df_train01.csv')
df_test.to_csv('../data/df_test01.csv')

df_chunk_train.to_csv('../data/df_chunk_train01.csv')
df_chunk_test.to_csv('../data/df_chunk_test01.csv')

In [35]:
df_all.to_csv('../data/df_all01.csv')
df_chunk.to_csv('../data/df_chunk01.csv')